In [1]:
#seeding for comparing experiment in part 2
import torch
SEED = 1111
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# import os
# os.environment["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device('cuda')

# SNLI Bert
This notebook has been copied from analytics_vidya_snli_medium.ipynb to investigate how encoding arguments together impacts performance.

In [2]:
# define macros
BERT_MODEL = 'bert-base-cased'
CORPUS = 'eng.rst.gum'
MAX_SEQ_LENGTH = 51
batches_per_epoch = 1700

# Prepare data

In [3]:
import pandas as pd
import csv

# we only need specific columns
train_df = pd.read_csv('./eng.rst.gum_train_enriched.rels', sep='\t', usecols = ['unit1_txt', 'unit1_sent', 'unit2_txt', 'unit2_sent', 'dir', 'label'])
val_df = pd.read_csv('./eng.rst.gum_test_enriched.rels', sep='\t', usecols = ['unit1_txt', 'unit1_sent', 'unit2_txt', 'unit2_sent', 'dir', 'label'])
test_df = pd.read_csv('./eng.rst.gum_dev_enriched.rels', sep='\t', usecols = ['unit1_txt', 'unit1_sent', 'unit2_txt', 'unit2_sent', 'dir', 'label'])

# /home/VD/kaveri/DisCoDisCo/gucorpling_models/rel/flair_clone_dataset_reader.py
# with open(rels_file_path, "r") as f:
#     reader = csv.DictReader(f, delimiter="\t", quoting=csv.QUOTE_NONE)

#     for i, row in enumerate(reader):
#         yield self.text_to_instance(
#             unit1_txt=row["unit1_txt"],
#             unit1_sent=row["unit1_sent"],
#             unit2_txt=row["unit2_txt"],
#             unit2_sent=row["unit2_sent"],
#             dir=row["dir"],
#             label=row["label"],
#             features=features[i]
#         )

train_df.head()

,unit1_txt,unit2_txt,unit1_sent,unit2_sent,dir,label
0,Aesthetic Appreciation and Spanish Art :,In this study we used eye-tracking in the firs...,Aesthetic Appreciation and Spanish Art :,In this study we used eye-tracking in the firs...,1>2,preparation
1,Aesthetic Appreciation and Spanish Art :,Insights from Eye-Tracking,Aesthetic Appreciation and Spanish Art :,Insights from Eye-Tracking,1<2,elaboration
2,Claire Bailey-Ross claire.bailey-ross@port.ac....,In this study we used eye-tracking in the firs...,Claire Bailey-Ross claire.bailey-ross@port.ac....,In this study we used eye-tracking in the firs...,1>2,attribution
3,Claire Bailey-Ross claire.bailey-ross@port.ac....,Andrew Beresford a.m.beresford@durham.ac.uk Du...,Claire Bailey-Ross claire.bailey-ross@port.ac....,Andrew Beresford a.m.beresford@durham.ac.uk Du...,1<2,joint
4,Claire Bailey-Ross claire.bailey-ross@port.ac....,Daniel Smith daniel.smith2@durham.ac.uk Durham...,Claire Bailey-Ross claire.bailey-ross@port.ac....,Daniel Smith daniel.smith2@durham.ac.uk Durham...,1<2,joint


## Clean the data

In [4]:
#dropping any empty values
train_df.dropna(inplace=True)
val_df.dropna(inplace=True)
test_df.dropna(inplace=True)

train_df = train_df[:int(len(train_df)/100)]
val_df = train_df[:int(len(val_df)/100)]
test_df = train_df[:int(len(test_df)/100)]

In [19]:
def get_labels(train_df, test_df, val_df):
    

labels = get_labels(train_df, test_df, val_df)



unit1_txt = train_df['unit1_txt'][0]
unit1_sent = train_df['unit1_sent'][0]
unit2_txt = train_df['unit2_txt'][0]
unit2_sent = train_df['unit2_sent'][0]
dir = train_df['dir'][0]
label = train_df['label'][0]

text_to_instance(mnli_dataset, unit1_txt, unit1_sent, unit2_txt, unit2_sent, dir, label, label_dict=labels, features = None)

{'combined_body': [101,
  198,
  13492,
  13542,
  1105,
  8492,
  2944,
  1893,
  131,
  135,
  102,
  1107,
  1142,
  2025,
  1195,
  1215,
  2552,
  118,
  10066,
  1107,
  1103,
  1148,
  2016,
  102],
 'combined_sentence': [101,
  198,
  13492,
  13542,
  1105,
  8492,
  2944,
  1893,
  131,
  135,
  102,
  1107,
  1142,
  2025,
  1195,
  1215,
  2552,
  118,
  10066,
  1107,
  1103,
  1148,
  2016,
  1104,
  12138,
  3703,
  2541,
  1104,
  1103,
  4154,
  8492,
  2944,
  1893,
  6286,
  1104,
  2514,
  3840,
  1197,
  2522,
  117,
  1104,
  1134,
  1103,
  1492,
  23199,
  18228,
  1179,
  25466,
  113,
  1175,
  1132,
  102],
 'direction': 1,
 'relation': 2}

## Prepare a dataset handler class

In [11]:
import numpy as np

def evaluate_accuracy(pred, target, prefered_target=None):
    num_examples, num_classes = target.shape

    correct = 0
    pred_refined = np.zeros_like(target)
    target_refined = np.zeros_like(target)
    # print(pred_refined)
    for i in range(num_examples):
        j = pred[i]
        pred_refined[i, j] = 1
        print(j)
        if target[i, j]:
            correct += 1
            target_refined[i, j] = 1
        else:
            if prefered_target is not None:
                j = prefered_target[i]
            else:
                j = target[i].argmax()
            target_refined[i, j] = 1

evaluate_accuracy(np.array([[0.5, 0.3, 0.2, 0.1, 0.0]]), np.array([[1, 0, 0, 0, 0]]))

IndexError: arrays used as indices must be of integer (or boolean) type

In [6]:

import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import BertTokenizer
import pandas as pd

class MNLIDataBert(Dataset):

  def __init__(self, train_df, val_df, test_df):
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    self.tokenizer = BertTokenizer.from_pretrained(BERT_MODEL, do_lower_case=True) # Using a pre-trained BERT tokenizer to encode sentences
    self.train_data = None
    self.val_data = None
    self.test_data = None
    # self.init_data()

  def init_data(self):
    self.get_label_mapping()
    self.train_data = self.load_data(self.train_df)
    self.val_data = self.load_data(self.val_df)
    self.test_data = self.load_data(self.test_df)

  def get_label_mapping(self):
    labels = {}
    labels_list = list(set(list(self.train_df['label'].unique()) + list(self.test_df['label'].unique()) + list(self.val_df['label'].unique())))
    for i in range(len(labels_list)):
        labels[labels_list[i]] = i
    self.label_dict = labels_list# {'entailment': 0, 'contradiction': 1, 'neutral': 2}
    # needed later for classification report object to generate precision and recall on test dataset
    self.rev_label_dict = {self.label_dict[k]:k for k in self.label_dict.keys()} 

  def text_to_instance(self, unit1_txt: str, unit1_sent: str, unit2_txt: str, unit2_sent: str, dir: str, label: str = None, label_dict = None, features = None):
    # line 34, /home/VD/kaveri/DisCoDisCo/gucorpling_models/rel/flair_clone_dataset_reader.py
    unit1_txt_tokens = self.tokenizer.tokenize(unit1_txt)
    unit1_sent_tokens = self.tokenizer.tokenize(unit1_sent)
    unit2_txt_tokens = self.tokenizer.tokenize(unit2_txt)
    unit2_sent_tokens = self.tokenizer.tokenize(unit2_sent)

    for tokens in [unit1_txt_tokens, unit1_sent_tokens, unit2_txt_tokens, unit2_sent_tokens]:
        tokens.insert(0, '[CLS]')
        tokens.append('[SEP]')
    cls_token = unit1_txt_tokens[0]
    sep_token = unit1_txt_tokens[-1]

    if cls_token not in ["[CLS]", '<s>']:
        raise Exception(f"Unrecognized cls token: {cls_token}")
    if sep_token not in ["[SEP]", '</s>']:
        raise Exception(f"Unrecognized sep token: {sep_token}")

    dir = '1>2'
    if dir == "1<2":
        left_tokens = []
        right_tokens = ["{"]
        dir_tokens = self.tokenizer.tokenize(sep_token + " <")#[0:-1]
        # print(left_tokens, right_tokens, dir_tokens, dir_tokens[0:-1])
    else:
        left_tokens = ["}"]
        right_tokens = []
        dir_tokens = self.tokenizer.tokenize("> " + sep_token)#[1:-1]
        # print(left_tokens, right_tokens, dir_tokens, dir_tokens[1:-1])
    # [CLS] } do we start ? > [SEP] no [SEP]
    # [CLS] thanks [SEP] < im ok { [SEP]
    combined_txt_tokens = (
        unit1_txt_tokens[:1]
        + left_tokens
        + unit1_txt_tokens[1:-1]
        + dir_tokens
        + unit2_txt_tokens[1:-1]
        + right_tokens
        + unit2_txt_tokens[-1:])[:MAX_SEQ_LENGTH]
    combined_sent_tokens = (
        unit1_sent_tokens[:1]
        + left_tokens
        + unit1_sent_tokens[1:-1]
        + dir_tokens
        + unit2_sent_tokens[1:-1]
        + right_tokens
        + unit2_sent_tokens[-1:]
    )[:MAX_SEQ_LENGTH]

    if dir == "1<2" and "{" not in combined_sent_tokens: combined_sent_tokens[-1] = "{"
    if dir == "1<2" and "{" not in combined_txt_tokens: combined_txt_tokens[-1] = "{"

    if combined_txt_tokens[-1]!='[SEP]': combined_txt_tokens.append('[SEP]')
    if combined_sent_tokens[-1]!='[SEP]': combined_sent_tokens.append('[SEP]')

    fields = {
        "combined_body": self.tokenizer.convert_tokens_to_ids(combined_txt_tokens),
        "combined_sentence": self.tokenizer.convert_tokens_to_ids(combined_sent_tokens),
        "direction": {"1<2":0, "1>2":1}[dir],
        }

    # # read in handcrafted features
    # if self.features is not None:
    #     features_configs = self.features.features

    #     for feature_name, feature_config in features_configs.items():
    #         if feature_name not in features:
    #             raise Exception(f"Feature {feature_name} not found. Pair:\n  {unit1_txt}\n  {unit2_txt}")
    #         if feature_name not in self.features.corpus_keys:
    #             continue
    #         feature_data = features[feature_name]
    #         fields[feature_name] = get_feature_field(feature_config, feature_data)

    if label:
        fields["relation"] = label_dict[label]
    return fields

  def load_data(self, df):
    MAX_LEN = 512 # dont need to enforce this now because snli is a sanitized dataset where sentence lenghts are reasonable. otherwise the beert model doesn't have enough parameters to handle long length sentences
    token_ids = []
    mask_ids = []
    seg_ids = []
    y = []

    premise_list = df['unit1_txt'].to_list()
    p_sent_list = df['unit1_sent'].to_list()
    hypothesis_list = df['unit2_txt'].to_list()
    h_sent_list = df['unit2_sent'].to_list()
    label_list = df['label'].to_list()

    for (premise, hypothesis, p_sent, h_sent, label) in zip(premise_list, hypothesis_list, p_sent_list, h_sent_list, label_list):
      # premise_id = self.tokenizer.encode(premise, add_special_tokens = False)
      # hypothesis_id = self.tokenizer.encode(hypothesis, add_special_tokens = False)
      # pair_token_ids = [self.tokenizer.cls_token_id] + premise_id + [self.tokenizer.sep_token_id] + hypothesis_id + [self.tokenizer.sep_token_id]
      # premise_len = len(premise_id)
      # hypothesis_len = len(hypothesis_id)
      fields = self.text_to_instance(premise, p_sent, hypothesis, h_sent, dir, label, self.label_dict, None)
      premise_hypothesis = fields['combined_sentence']

      segment_ids = torch.tensor([0] * (premise_len + 2) + [1] * (hypothesis_len + 1))  # sentence 0 and sentence 1
      attention_mask_ids = torch.tensor([1] * (premise_len + hypothesis_len + 3))  # mask padded values

      token_ids.append(torch.tensor(pair_token_ids))
      seg_ids.append(segment_ids)
      mask_ids.append(attention_mask_ids)
      y.append(self.label_dict[label])
    
    token_ids = pad_sequence(token_ids, batch_first=True)
    mask_ids = pad_sequence(mask_ids, batch_first=True)
    seg_ids = pad_sequence(seg_ids, batch_first=True)
    print('token_ids: ', token_ids[2])
    print('seg_ids: ', seg_ids[2])
    print('mask_ids: ', mask_ids[2])

    y = torch.tensor(y)
    dataset = TensorDataset(token_ids, mask_ids, seg_ids, y)
    return dataset

  def get_data_loaders(self, batch_size=32, shuffle=True):
    train_loader = DataLoader(
      self.train_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    val_loader = DataLoader(
      self.val_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    test_loader = DataLoader(
      self.test_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    return train_loader, val_loader, test_loader

In [7]:
mnli_dataset = MNLIDataBert(train_df, val_df, test_df)

# train_loader, val_loader, test_loader = mnli_dataset.get_data_loaders()
# label_dict = mnli_dataset.label_dict # required by custom func to calculate accuracy
# rev_label_dict = mnli_dataset.rev_label_dict # required by custom func to calculate accuracy

In [9]:
from transformers import BertForSequenceClassification, AdamW

model = BertForSequenceClassification.from_pretrained(BERT_MODEL, num_labels=3).to(device)
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [11]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [12]:
# to evaluate model for train and test. And also use classification report for testing

# helper function to calculate the batch accuracy
def multi_acc(y_pred, y_test):
  acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test).sum().float() / float(y_test.size(0))
  return acc

# freeze model weights and measure validation / test 
def evaluate_accuracy(model, optimizer, data_loader, rev_label_dict):
  model.eval()
  total_val_acc  = 0
  total_val_loss = 0
  
  #for classification report
  y_true = []
  y_pred = []
  
  with torch.no_grad():
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(data_loader):
      optimizer.zero_grad()
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)
      
      loss, prediction = model(pair_token_ids, 
                            token_type_ids=seg_ids, 
                            attention_mask=mask_ids, 
                            labels=labels).values()
      
      acc = multi_acc(prediction, labels)

      total_val_loss += loss.item()
      total_val_acc  += acc.item()

      argmax_predictions = torch.argmax(prediction,dim=1).tolist()
      labels_list = labels.tolist()
      assert(len(labels_list)==len(argmax_predictions))
      for p in argmax_predictions: y_pred.append(rev_label_dict[int(p)])
      for l in labels_list: y_true.append(rev_label_dict[l])

  val_acc  = total_val_acc/len(data_loader)
  val_loss = total_val_loss/len(data_loader)
  cr = classification_report(y_true, y_pred)
  
  return val_acc, val_loss, cr, model, optimizer

In [13]:
import time
from sklearn.metrics import classification_report

EPOCHS = 5

def train(model, train_loader, val_loader, optimizer, rev_label_dict):  
  for epoch in range(EPOCHS):
    start = time.time()
    model.train()
    total_train_loss = 0
    total_train_acc  = 0
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(train_loader):
      optimizer.zero_grad()
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)

      loss, prediction = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels).values()

      acc = multi_acc(prediction, labels)

      loss.backward()
      optimizer.step()
      
      total_train_loss += loss.item()
      total_train_acc  += acc.item()

    train_acc  = total_train_acc/len(train_loader)
    train_loss = total_train_loss/len(train_loader)

    val_acc, val_loss, cr, model, optimizer = evaluate_accuracy(model, optimizer, val_loader, rev_label_dict)
    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)

    print(f'Epoch {epoch+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [14]:
train(model, train_loader, val_loader, optimizer, rev_label_dict)

Epoch 1: train_loss: 0.8215 train_acc: 0.6229 | val_loss: 0.3005 val_acc: 0.9062
00:01:08.58
Epoch 2: train_loss: 0.4633 train_acc: 0.8307 | val_loss: 0.2943 val_acc: 0.9453
00:01:07.73
Epoch 3: train_loss: 0.2784 train_acc: 0.9073 | val_loss: 0.0823 val_acc: 0.9844
00:01:07.75
Epoch 4: train_loss: 0.1557 train_acc: 0.9539 | val_loss: 0.0491 val_acc: 0.9922
00:01:07.69
Epoch 5: train_loss: 0.1111 train_acc: 0.9664 | val_loss: 0.0103 val_acc: 1.0000
00:01:07.70


In [ ]:
# torch.save(model.state_dict(), 'bert-nli-.pt')

In [ ]:
def validate(model, test_loader, optimizer, rev_label_dict):
  start = time.time()
  test_acc, test_loss, cr, model, optimizer = evaluate_accuracy(model, optimizer, test_loader, rev_label_dict)
  end = time.time()
  hours, rem = divmod(end-start, 3600)
  minutes, seconds = divmod(rem, 60)

  print(f'Test_loss: {test_loss:.4f} test_acc: {test_acc:.4f}')
  print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
  print(cr)

  return test_loss, test_acc


# model.load_state_dict(torch.load('bert-nli.pt'))
test_loss, test_acc = validate(model, test_loader, optimizer, rev_label_dict)
print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

Test_loss: 0.8967 test_acc: 0.5625
00:00:00.12
               precision    recall  f1-score   support

contradiction       0.91      0.30      0.45        33
   entailment       0.75      0.83      0.79        36
      neutral       0.51      0.81      0.62        31

     accuracy                           0.65       100
    macro avg       0.72      0.65      0.62       100
 weighted avg       0.73      0.65      0.63       100

Test Loss: 0.897 |  Test Acc: 56.25%


In [ ]:
# TODO: Medium page.
# TODO: Make code modular. eg - evaluation called for dev and test adn train. Loss and prediction can become loss and logits. Another wrapper to return basic predictions/log them in it as well.